# base

> Base classes, types, and protocols for Tailwind CSS abstractions

In [ ]:
#| default_exp core.base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Union, Optional, Literal, Protocol, runtime_checkable, TypeVar, Generic, Callable, Any, Dict, List, Tuple
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum
import re

## Core Types

Define the fundamental types used throughout the library:

In [ ]:
#| export
# Type aliases for Tailwind values
TailwindScale = Union[int, float, str]  # Numeric scales like 1, 2.5, "px"
TailwindFraction = str  # Fractions like "1/2", "2/3"
TailwindArbitrary = str  # Arbitrary values like "123px", "10rem"
TailwindCustomProperty = str  # CSS custom properties like "--spacing-lg"
TailwindValue = Union[TailwindScale, TailwindFraction, TailwindArbitrary, TailwindCustomProperty] # Union of all possible value types

## Value Validators

Functions to validate and identify different types of Tailwind values:

In [ ]:
#| export
def is_numeric_scale(
    value: Any  # The value to check - can be int, float, or string
) -> bool:  # True if the value is a valid numeric scale, False otherwise
    """Check if value is a valid numeric scale (int, float, or 'px')."""
    if isinstance(value, (int, float)):
        return True
    if isinstance(value, str) and value == "px":
        return True
    return False

In [ ]:
#| export
def is_fraction(
    value: Any  # The value to check for fraction format
) -> bool:  # True if the value is a valid fraction string, False otherwise
    """Check if value is a valid fraction string (e.g., '1/2', '3/4')."""
    if not isinstance(value, str):
        return False
    pattern = r'^\d+/\d+$'
    return bool(re.match(pattern, value))

In [ ]:
#| export
def is_custom_property(
    value: Any  # The value to check for CSS custom property format
) -> bool:  # True if the value is a CSS custom property, False otherwise
    """Check if value is a CSS custom property (starts with --)."""
    if not isinstance(value, str):
        return False
    return value.startswith("--")

In [ ]:
#| export
def is_arbitrary_value(
    value: Any  # The value to check for arbitrary CSS value format
) -> bool:  # True if the value contains CSS units or calc(), False otherwise
    """Check if value is an arbitrary value (contains units or special chars)."""
    if not isinstance(value, str):
        return False
    # Check for common CSS units
    units = r'(px|em|rem|vh|vw|%|deg|s|ms)$'
    return bool(re.search(units, value)) or value.startswith("calc(")

## Base Protocol

Define the protocol that all utility builders must implement:

In [ ]:
#| export
@runtime_checkable
class TailwindBuilder(Protocol):
    """Protocol for all Tailwind utility builders."""
    
    def build(
        self,
        *args,
        **kwargs
    ) -> str:  # The built CSS class string
        """Build and return the CSS class string."""
        ...
    
    def __str__(
        self
    ) -> str:  # The built CSS class string
        """Return the built CSS class string."""
        ...

## Base Classes

Abstract base classes for different types of utility builders:

In [ ]:
#| export
class BaseUtility(ABC):
    """Base class for all Tailwind utility builders."""
    
    def __init__(
        self,
        prefix: str  # The utility prefix (e.g., 'w' for width, 'p' for padding)
    ):
        """
        Initialize with a utility prefix.
        """
        self.prefix = prefix
        self._value: Optional[str] = None
        self._modifiers: List[str] = []
    
    @abstractmethod
    def _format_value(
        self,
        value: TailwindValue  # The value to format (numeric, fraction, arbitrary, etc.)
    ) -> str:  # The formatted value string ready for class construction
        """Format the value according to Tailwind conventions."""
        pass
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use for building the class
    ) -> str:  # The complete CSS class string with modifiers applied
        """Build the complete CSS class string."""
        if value is not None:
            formatted_value = self._format_value(value)
        elif self._value is not None:
            formatted_value = self._value
        else:
            return self.prefix
        
        # Join prefix and value
        base_class = f"{self.prefix}-{formatted_value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class
    
    def build(
        self,
        value: Optional[TailwindValue] = None  # Optional value to override the stored value
    ) -> str:  # The built CSS class string
        """Build and return the CSS class string."""
        return self._build_class(value)
    
    def __str__(
        self
    ) -> str:  # The built CSS class string
        """Return the built CSS class string."""
        return self._build_class()
    
    def with_modifiers(
        self,
        *modifiers: str  # Modifier strings to apply (e.g., 'hover', 'focus', 'dark')
    ) -> 'BaseUtility':  # A new instance with the modifiers applied
        """
        Create a new instance with additional modifiers.
        Modifiers are applied in the order they are passed.
        """
        # Create a shallow copy to avoid modifying the original
        import copy
        new_instance = copy.copy(self)
        new_instance._modifiers = list(modifiers) + self._modifiers
        return new_instance

## Standard Value Formatter

A standard implementation for formatting Tailwind values:

In [ ]:
#| export
class ModifierMixin:
    """Mixin to add modifier support to any utility with convenient property access."""
    
    # Pseudo-class modifiers
    @property
    def hover(self) -> 'BaseUtility':
        """Apply hover modifier."""
        return self.with_modifiers("hover")
    
    @property
    def focus(self) -> 'BaseUtility':
        """Apply focus modifier."""
        return self.with_modifiers("focus")
    
    @property
    def active(self) -> 'BaseUtility':
        """Apply active modifier."""
        return self.with_modifiers("active")
    
    @property
    def visited(self) -> 'BaseUtility':
        """Apply visited modifier."""
        return self.with_modifiers("visited")
    
    @property
    def disabled(self) -> 'BaseUtility':
        """Apply disabled modifier."""
        return self.with_modifiers("disabled")
    
    @property
    def checked(self) -> 'BaseUtility':
        """Apply checked modifier."""
        return self.with_modifiers("checked")
    
    @property
    def required(self) -> 'BaseUtility':
        """Apply required modifier."""
        return self.with_modifiers("required")
    
    @property
    def invalid(self) -> 'BaseUtility':
        """Apply invalid modifier."""
        return self.with_modifiers("invalid")
    
    @property
    def valid(self) -> 'BaseUtility':
        """Apply valid modifier."""
        return self.with_modifiers("valid")
    
    # Pseudo-element modifiers
    @property
    def before(self) -> 'BaseUtility':
        """Apply before pseudo-element modifier."""
        return self.with_modifiers("before")
    
    @property
    def after(self) -> 'BaseUtility':
        """Apply after pseudo-element modifier."""
        return self.with_modifiers("after")
    
    @property
    def placeholder(self) -> 'BaseUtility':
        """Apply placeholder modifier."""
        return self.with_modifiers("placeholder")
    
    @property
    def selection(self) -> 'BaseUtility':
        """Apply selection modifier."""
        return self.with_modifiers("selection")
    
    # Responsive modifiers
    @property
    def sm(self) -> 'BaseUtility':
        """Apply small breakpoint modifier."""
        return self.with_modifiers("sm")
    
    @property
    def md(self) -> 'BaseUtility':
        """Apply medium breakpoint modifier."""
        return self.with_modifiers("md")
    
    @property
    def lg(self) -> 'BaseUtility':
        """Apply large breakpoint modifier."""
        return self.with_modifiers("lg")
    
    @property
    def xl(self) -> 'BaseUtility':
        """Apply extra large breakpoint modifier."""
        return self.with_modifiers("xl")
    
    @property
    def _2xl(self) -> 'BaseUtility':
        """Apply 2xl breakpoint modifier."""
        return self.with_modifiers("2xl")
    
    # Theme modifiers
    @property
    def dark(self) -> 'BaseUtility':
        """Apply dark mode modifier."""
        return self.with_modifiers("dark")
    
    # Motion modifiers
    @property
    def motion_reduce(self) -> 'BaseUtility':
        """Apply reduced motion modifier."""
        return self.with_modifiers("motion-reduce")
    
    @property
    def motion_safe(self) -> 'BaseUtility':
        """Apply safe motion modifier."""
        return self.with_modifiers("motion-safe")
    
    # Structural modifiers
    @property
    def first(self) -> 'BaseUtility':
        """Apply first child modifier."""
        return self.with_modifiers("first")
    
    @property
    def last(self) -> 'BaseUtility':
        """Apply last child modifier."""
        return self.with_modifiers("last")
    
    @property
    def odd(self) -> 'BaseUtility':
        """Apply odd child modifier."""
        return self.with_modifiers("odd")
    
    @property
    def even(self) -> 'BaseUtility':
        """Apply even child modifier."""
        return self.with_modifiers("even")
    
    # Group and peer modifiers
    def group(
        self, 
        state: Optional[str] = None,  # Optional state like 'hover', 'focus'
        name: Optional[str] = None    # Optional group name for nested groups
    ) -> 'BaseUtility':  # The utility with group modifier applied
        """Apply group modifier with optional state and name."""
        modifier = "group"
        if name:
            modifier = f"group/{name}"
        if state:
            modifier = f"{modifier}-{state}"
        return self.with_modifiers(modifier)
    
    def peer(
        self, 
        state: Optional[str] = None,  # Optional state like 'hover', 'focus'
        name: Optional[str] = None    # Optional peer name for multiple peers
    ) -> 'BaseUtility':  # The utility with peer modifier applied
        """Apply peer modifier with optional state and name."""
        modifier = "peer"
        if name:
            modifier = f"peer/{name}"
        if state:
            modifier = f"{modifier}-{state}"
        return self.with_modifiers(modifier)
    
    # Arbitrary modifiers
    def has(
        self,
        selector: str  # CSS selector for :has() pseudo-class
    ) -> 'BaseUtility':  # The utility with has modifier applied
        """Apply has modifier with a selector."""
        return self.with_modifiers(f"has-[{selector}]")
    
    def aria(
        self,
        attribute: str,  # ARIA attribute name
        value: Optional[str] = None  # Optional value for the attribute
    ) -> 'BaseUtility':  # The utility with aria modifier applied
        """Apply aria modifier with attribute and optional value."""
        if value:
            return self.with_modifiers(f"aria-[{attribute}={value}]")
        return self.with_modifiers(f"aria-{attribute}")
    
    def data(
        self,
        attribute: str,  # Data attribute name
        value: Optional[str] = None  # Optional value for the attribute
    ) -> 'BaseUtility':  # The utility with data modifier applied
        """Apply data modifier with attribute and optional value."""
        if value:
            return self.with_modifiers(f"data-[{attribute}={value}]")
        return self.with_modifiers(f"data-[{attribute}]")
    
    def arbitrary(
        self,
        selector: str  # Arbitrary CSS selector
    ) -> 'BaseUtility':  # The utility with arbitrary modifier applied
        """Apply arbitrary modifier with custom selector."""
        return self.with_modifiers(f"[{selector}]")

## Modifier Definitions

Comprehensive definitions of all Tailwind CSS modifiers/variants:

In [ ]:
#| export
@dataclass
class ModifierGroup:
    """Group of related modifiers with descriptions."""
    name: str
    description: str
    modifiers: Dict[str, str]  # modifier_name -> tailwind_variant

# Pseudo-class modifiers
PSEUDO_CLASS_MODIFIERS = ModifierGroup(
    "Pseudo Classes",
    "Style elements based on pseudo-class states",
    {
        # Interactive states
        "hover": "hover",
        "focus": "focus",
        "focus_within": "focus-within",
        "focus_visible": "focus-visible",
        "active": "active",
        "visited": "visited",
        "target": "target",
        
        # Form states
        "disabled": "disabled",
        "enabled": "enabled",
        "checked": "checked",
        "indeterminate": "indeterminate",
        "default": "default",
        "required": "required",
        "valid": "valid",
        "invalid": "invalid",
        "in_range": "in-range",
        "out_of_range": "out-of-range",
        "placeholder_shown": "placeholder-shown",
        "autofill": "autofill",
        "read_only": "read-only",
        
        # Structural states
        "first": "first",
        "last": "last",
        "only": "only",
        "odd": "odd",
        "even": "even",
        "first_of_type": "first-of-type",
        "last_of_type": "last-of-type",
        "only_of_type": "only-of-type",
        "empty": "empty",
    }
)

# Pseudo-element modifiers
PSEUDO_ELEMENT_MODIFIERS = ModifierGroup(
    "Pseudo Elements",
    "Style pseudo-elements of an element",
    {
        "before": "before",
        "after": "after",
        "first_letter": "first-letter",
        "first_line": "first-line",
        "marker": "marker",
        "selection": "selection",
        "file": "file",
        "backdrop": "backdrop",
        "placeholder": "placeholder",
    }
)

# Responsive modifiers
RESPONSIVE_MODIFIERS = ModifierGroup(
    "Responsive",
    "Apply styles at specific breakpoints",
    {
        "sm": "sm",      # >= 640px
        "md": "md",      # >= 768px
        "lg": "lg",      # >= 1024px
        "xl": "xl",      # >= 1280px
        "2xl": "2xl",    # >= 1536px
    }
)

# Dark mode and theme modifiers
THEME_MODIFIERS = ModifierGroup(
    "Theme",
    "Apply styles based on color scheme preference",
    {
        "dark": "dark",
        "light": "light",  # For explicit light mode styling
    }
)

# Motion modifiers
MOTION_MODIFIERS = ModifierGroup(
    "Motion",
    "Apply styles based on motion preferences",
    {
        "motion_safe": "motion-safe",
        "motion_reduce": "motion-reduce",
    }
)

# Print modifier
PRINT_MODIFIERS = ModifierGroup(
    "Print",
    "Apply styles for print media",
    {
        "print": "print",
    }
)

# Orientation modifiers
ORIENTATION_MODIFIERS = ModifierGroup(
    "Orientation",
    "Apply styles based on viewport orientation",
    {
        "portrait": "portrait",
        "landscape": "landscape",
    }
)

# Contrast modifiers
CONTRAST_MODIFIERS = ModifierGroup(
    "Contrast",
    "Apply styles based on contrast preference",
    {
        "contrast_more": "contrast-more",
        "contrast_less": "contrast-less",
    }
)

# Direction modifiers
DIRECTION_MODIFIERS = ModifierGroup(
    "Direction",
    "Apply styles based on text direction",
    {
        "rtl": "rtl",
        "ltr": "ltr",
    }
)

# Open/closed state modifiers
STATE_MODIFIERS = ModifierGroup(
    "State",
    "Apply styles based on open/closed states",
    {
        "open": "open",
        "closed": "closed",
    }
)

# Child selector modifiers
CHILD_MODIFIERS = ModifierGroup(
    "Children",
    "Apply styles to child elements",
    {
        "children": "*",      # Direct children
        "descendants": "**",  # All descendants
    }
)

# All modifier groups
ALL_MODIFIER_GROUPS = [
    PSEUDO_CLASS_MODIFIERS,
    PSEUDO_ELEMENT_MODIFIERS,
    RESPONSIVE_MODIFIERS,
    THEME_MODIFIERS,
    MOTION_MODIFIERS,
    PRINT_MODIFIERS,
    ORIENTATION_MODIFIERS,
    CONTRAST_MODIFIERS,
    DIRECTION_MODIFIERS,
    STATE_MODIFIERS,
    CHILD_MODIFIERS,
]

In [ ]:
#| export
class StandardUtility(BaseUtility, ModifierMixin):
    """Standard utility class with common value formatting and modifier support."""
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format according to Tailwind conventions
    ) -> str:  # The formatted value string (e.g., "4", "[10px]", "(--custom)")
        """
        Format value according to Tailwind conventions:
        - Numeric scales: used as-is (e.g., 4 -> "4")
        - Fractions: used as-is (e.g., "1/2" -> "1/2")
        - Custom properties: wrapped in parentheses (e.g., "--spacing" -> "(--spacing)")
        - Arbitrary values: wrapped in brackets (e.g., "10px" -> "[10px]")
        """
        if is_numeric_scale(value):
            return str(value)
        elif is_fraction(value):
            return value
        elif is_custom_property(value):
            return f"({value})"
        elif is_arbitrary_value(value):
            return f"[{value}]"
        else:
            # Named values (like 'auto', 'full', etc.)
            return str(value)

## Named Scale Utilities

For utilities that support named scales (e.g., sm, md, lg):

In [ ]:
#| export
@dataclass
class NamedScale:
    """Represents a named scale with optional CSS variable."""
    name: str
    var: Optional[str] = None
    comment: Optional[str] = None
    
    def format(
        self
    ) -> str:  # The name of the scale for use in CSS classes
        """Format as Tailwind class suffix."""
        return self.name

In [ ]:
#| export
CONTAINER_SCALES = [ # Common named scales used across utilities
    NamedScale("3xs", "--container-3xs", "16rem (256px)"),
    NamedScale("2xs", "--container-2xs", "18rem (288px)"),
    NamedScale("xs", "--container-xs", "20rem (320px)"),
    NamedScale("sm", "--container-sm", "24rem (384px)"),
    NamedScale("md", "--container-md", "28rem (448px)"),
    NamedScale("lg", "--container-lg", "32rem (512px)"),
    NamedScale("xl", "--container-xl", "36rem (576px)"),
    NamedScale("2xl", "--container-2xl", "42rem (672px)"),
    NamedScale("3xl", "--container-3xl", "48rem (768px)"),
    NamedScale("4xl", "--container-4xl", "56rem (896px)"),
    NamedScale("5xl", "--container-5xl", "64rem (1024px)"),
    NamedScale("6xl", "--container-6xl", "72rem (1152px)"),
    NamedScale("7xl", "--container-7xl", "80rem (1280px)"),
]

## Modifier Support

Support for responsive and state modifiers:

In [ ]:
#| export
@dataclass
class Breakpoint:
    """Responsive breakpoint definition."""
    name: str
    min_width: Optional[str] = None

In [ ]:
#| export
BREAKPOINTS = { # Common breakpoints
    "sm": Breakpoint("sm", "640px"),
    "md": Breakpoint("md", "768px"),
    "lg": Breakpoint("lg", "1024px"),
    "xl": Breakpoint("xl", "1280px"),
    "2xl": Breakpoint("2xl", "1536px"),
}

In [ ]:
#| export
STATE_MODIFIERS = [ # Common state modifiers
    "hover", "focus", "active", "visited", "target",
    "focus-within", "focus-visible", "disabled", "enabled",
    "checked", "indeterminate", "default", "required",
    "valid", "invalid", "in-range", "out-of-range",
    "placeholder-shown", "autofill", "read-only",
    "first", "last", "odd", "even", "first-of-type",
    "last-of-type", "only-child", "empty",
    "before", "after", "first-letter", "first-line",
    "marker", "selection", "file", "backdrop",
    "placeholder", "open", "closed",
]

## Base Factory with Documentation

A base factory class that all factories inherit from, providing documentation support:

In [ ]:
#| export
class BaseFactory(ABC):
    """Base factory class with documentation support."""
    
    def __init__(
        self,
        doc: str  # Documentation string describing what this factory creates
    ):
        """Initialize with documentation string."""
        self._doc = doc
    
    @property
    def __doc__(
        self
    ) -> str:  # The documentation string
        """Return the documentation for this factory."""
        return self._doc
    
    def describe(
        self
    ) -> str:  # A formatted description of the factory
        """Return a formatted description of this factory."""
        return self._doc
    
    @abstractmethod
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """
        Get detailed information about this factory's options and valid inputs.
        
        Should return a dictionary with keys like:
        - 'description': Factory description
        - 'valid_inputs': List/description of valid input values
        - 'options': Available options or methods
        """
        pass

## Utility Factory

A factory function to create utility instances with method chaining:

In [ ]:
#| export
T = TypeVar('T', bound=BaseUtility)

In [ ]:
#| export
class UtilityFactory(BaseFactory, Generic[T]):
    """Factory for creating utility instances with fluent API."""
    
    def __init__(
        self,
        utility_class: type[T],  # The utility class to instantiate
        prefix: str,  # The prefix to use for the utilities
        doc: Optional[str] = None  # Optional documentation string
    ):
        "Initialize factory with a utility class and prefix."
        doc = doc or f"Factory for {prefix} utilities"
        super().__init__(doc)
        self.utility_class = utility_class
        self.prefix = prefix
    
    def __call__(
        self,
        value: Optional[TailwindValue] = None  # Initial value for the utility
    ) -> T:  # A new instance of the utility class
        """Create a utility instance with optional value."""
        instance = self.utility_class(self.prefix)
        if value is not None:
            instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(
        self,
        name: str  # Attribute name to convert to a utility value
    ) -> T:  # A new utility instance with the attribute as its value
        """Handle named values (e.g., w.full, h.screen)."""
        instance = self.utility_class(self.prefix)
        instance._value = name.replace("_", "-")
        return instance
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about this utility factory."""
        return {
            'description': self._doc,
            'valid_inputs': 'Various Tailwind values (implementation specific)',
            'options': {
                'prefix': self.prefix,
                'utility_class': self.utility_class.__name__
            }
        }

## Examples

Let's test the base architecture with some examples:

In [ ]:
# Test numeric scale
assert is_numeric_scale(4) == True
assert is_numeric_scale(2.5) == True
assert is_numeric_scale("px") == True
assert is_numeric_scale("auto") == False

In [ ]:
# Test fraction detection
assert is_fraction("1/2") == True
assert is_fraction("3/4") == True
assert is_fraction("1.5") == False
assert is_fraction("full") == False

In [ ]:
# Test custom property detection
assert is_custom_property("--spacing-lg") == True
assert is_custom_property("--color-primary") == True
assert is_custom_property("spacing-lg") == False

In [ ]:
# Test arbitrary value detection
assert is_arbitrary_value("10px") == True
assert is_arbitrary_value("2.5rem") == True
assert is_arbitrary_value("100%") == True
assert is_arbitrary_value("calc(100% - 20px)") == True
assert is_arbitrary_value("auto") == False

In [ ]:
# Test StandardUtility class
class TestUtility(StandardUtility):
    pass

# Create test instances
util = TestUtility("w")

# Test different value types
assert util.build(4) == "w-4"
assert util.build("1/2") == "w-1/2"
assert util.build("--custom") == "w-(--custom)"
assert util.build("10px") == "w-[10px]"
assert util.build("auto") == "w-auto"

In [ ]:
# Test UtilityFactory
w = UtilityFactory(TestUtility, "w")

# Test factory patterns
assert str(w(4)) == "w-4"
assert str(w.full) == "w-full"
assert str(w.auto) == "w-auto"
assert str(w.screen) == "w-screen"

### Test Modifiers

Test the modifier support on utilities:

In [ ]:
# First, let's update TestUtility to inherit from StandardUtility instead
class TestUtilityWithModifiers(StandardUtility):
    pass

# Test basic modifier application
util = TestUtilityWithModifiers("bg")
util._value = "red-500"

# Test single modifiers
assert str(util.hover) == "hover:bg-red-500"
assert str(util.focus) == "focus:bg-red-500"
assert str(util.dark) == "dark:bg-red-500"

# Test chained modifiers
assert str(util.hover.dark) == "dark:hover:bg-red-500"
assert str(util.md.hover) == "hover:md:bg-red-500"
assert str(util.sm.hover.dark) == "dark:hover:sm:bg-red-500"

# Test responsive modifiers
assert str(util.sm) == "sm:bg-red-500"
assert str(util.md) == "md:bg-red-500"
assert str(util.lg) == "lg:bg-red-500"
assert str(util._2xl) == "2xl:bg-red-500"

# Test pseudo-element modifiers
assert str(util.before) == "before:bg-red-500"
assert str(util.after) == "after:bg-red-500"
assert str(util.placeholder) == "placeholder:bg-red-500"

# Test structural modifiers
assert str(util.first) == "first:bg-red-500"
assert str(util.last) == "last:bg-red-500"
assert str(util.odd) == "odd:bg-red-500"
assert str(util.even) == "even:bg-red-500"

print("✅ Basic modifier tests passed!")

✅ Basic modifier tests passed!


In [ ]:
# Test group and peer modifiers
assert str(util.group()) == "group:bg-red-500"
assert str(util.group("hover")) == "group-hover:bg-red-500"
assert str(util.group("focus", "sidebar")) == "group/sidebar-focus:bg-red-500"

assert str(util.peer()) == "peer:bg-red-500"
assert str(util.peer("checked")) == "peer-checked:bg-red-500"
assert str(util.peer("invalid", "email")) == "peer/email-invalid:bg-red-500"

# Test arbitrary modifiers
assert str(util.has("input:checked")) == "has-[input:checked]:bg-red-500"
assert str(util.aria("checked")) == "aria-checked:bg-red-500"
assert str(util.aria("sort", "ascending")) == "aria-[sort=ascending]:bg-red-500"
assert str(util.data("active")) == "data-[active]:bg-red-500"
assert str(util.data("state", "open")) == "data-[state=open]:bg-red-500"
assert str(util.arbitrary("&.is-dragging")) == "[&.is-dragging]:bg-red-500"

print("✅ Group/peer and arbitrary modifier tests passed!")

✅ Group/peer and arbitrary modifier tests passed!


## Class Combination Utility

A utility function to combine multiple class builders:

In [ ]:
#| export
def combine_classes(
    *args: Union[str, BaseUtility, TailwindBuilder, BaseFactory, None]
) -> str:  # Space-separated class string
    "Combine multiple class builders or strings into a single class string."
    classes = []
    for arg in args:
        if arg is None:
            continue
        elif isinstance(arg, str):
            if arg.strip():  # Only add non-empty strings
                classes.append(arg.strip())
        elif isinstance(arg, BaseUtility):
            classes.append(str(arg))
        elif isinstance(arg, SingleValueFactory):
            classes.append(str(arg))
        elif hasattr(arg, 'build'):
            classes.append(arg.build())
        elif hasattr(arg, '__str__'):
            result = str(arg)
            if result.strip():
                classes.append(result)
    
    return " ".join(classes)

In [ ]:
# Test combine_classes
w_factory = UtilityFactory(TestUtility, "w")
h_factory = UtilityFactory(TestUtility, "h")

result = combine_classes(
    w_factory(4),
    h_factory.full,
    "flex",
    None,
    "items-center"
)
assert result == "w-4 h-full flex items-center"

In [ ]:
# Test combine_classes with modifiers
w_hover = TestUtilityWithModifiers("w")
w_hover._value = "full"
w_hover = w_hover.hover

h_dark = TestUtilityWithModifiers("h")
h_dark._value = "screen"
h_dark = h_dark.dark.md

result = combine_classes(
    w_hover,
    h_dark,
    "flex",
    None,
    "items-center"
)
assert result == "hover:w-full md:dark:h-screen flex items-center"

print("✅ combine_classes with modifiers test passed!")

✅ combine_classes with modifiers test passed!


## Practical Examples with FastHTML

Comprehensive examples showing modifiers in action:

In [ ]:
# Example: Interactive button with hover, focus, and active states
from fasthtml.common import Button

# Create a utility instance with modifiers
bg_base = TestUtilityWithModifiers("bg")
bg_base._value = "blue-500"

bg_hover = TestUtilityWithModifiers("bg")
bg_hover._value = "blue-600"

bg_active = TestUtilityWithModifiers("bg")
bg_active._value = "blue-700"

# Create button with multiple states
button = Button(
    "Click me",
    cls=combine_classes(
        bg_base,
        bg_hover.hover,
        bg_active.active,
        "text-white px-4 py-2 rounded"
    )
)

assert button.attrs['class'] == "bg-blue-500 hover:bg-blue-600 active:bg-blue-700 text-white px-4 py-2 rounded"
print("✅ Interactive button example passed!")

✅ Interactive button example passed!


In [ ]:
# Example: Responsive layout with dark mode support
from fasthtml.common import Div, H1, P

# Create utilities with responsive and dark mode modifiers
padding = TestUtilityWithModifiers("p")
padding._value = "4"

padding_md = TestUtilityWithModifiers("p")
padding_md._value = "8"

bg_light = TestUtilityWithModifiers("bg")
bg_light._value = "white"

bg_dark = TestUtilityWithModifiers("bg")
bg_dark._value = "gray-900"

text_light = TestUtilityWithModifiers("text")
text_light._value = "gray-900"

text_dark = TestUtilityWithModifiers("text")
text_dark._value = "white"

# Create responsive card component
card = Div(
    H1("Responsive Card", cls=combine_classes(text_light, text_dark.dark)),
    P("This card adapts to screen size and color scheme."),
    cls=combine_classes(
        padding,
        padding_md.md,
        bg_light,
        bg_dark.dark,
        "rounded-lg shadow"
    )
)

assert card.attrs['class'] == "p-4 md:p-8 bg-white dark:bg-gray-900 rounded-lg shadow"
assert card.children[0].attrs['class'] == "text-gray-900 dark:text-white"
print("✅ Responsive dark mode example passed!")

✅ Responsive dark mode example passed!


In [ ]:
# Example: Form input with various states
from fasthtml.common import Input, Label, Span

# Create utilities for form states
border_base = TestUtilityWithModifiers("border")
border_base._value = "gray-300"

border_focus = TestUtilityWithModifiers("border")
border_focus._value = "blue-500"

border_invalid = TestUtilityWithModifiers("border")
border_invalid._value = "red-500"

# Create form input with label
form_field = Label(
    Span("Email", cls="block text-sm font-medium mb-1"),
    Input(
        type="email",
        required=True,
        cls=combine_classes(
            "w-full px-3 py-2 rounded-md",
            border_base,
            border_focus.focus,
            border_invalid.invalid,
            "outline-none focus:ring-2 focus:ring-blue-500/20"
        )
    )
)

assert "border-gray-300" in form_field.children[1].attrs['class']
assert "focus:border-blue-500" in form_field.children[1].attrs['class']
assert "invalid:border-red-500" in form_field.children[1].attrs['class']
print("✅ Form states example passed!")

✅ Form states example passed!


## Single Value Factory

A factory for standalone utility class strings:

In [ ]:
#| export
class SingleValueFactory(BaseFactory):
    """Factory for a single utility class string with documentation."""
    
    def __init__(
        self,
        value: str,  # The utility class string (e.g., "container")
        doc: str  # Documentation describing what this utility does
    ):
        """Initialize with a value and documentation."""
        super().__init__(doc)
        self._value = value
    
    def __str__(
        self
    ) -> str:  # The utility class string
        """Return the utility class string."""
        return self._value
    
    def __call__(
        self
    ) -> str:  # The utility class string
        """Return the utility class string when called."""
        return self._value
    
    def build(
        self
    ) -> str:  # The utility class string
        """Build and return the utility class string."""
        return self._value
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about this single-value factory."""
        return {
            'description': self._doc,
            'valid_inputs': 'No inputs - returns a fixed value',
            'options': {
                'value': self._value
            }
        }

In [ ]:
# Test SingleValueFactory
container = SingleValueFactory(
    "container", 
    "Responsive container with breakpoint-based max-widths"
)

# Test various ways to use it
assert str(container) == "container"
assert container() == "container"
assert container.build() == "container"
assert container.describe() == "Responsive container with breakpoint-based max-widths"

# Test in combine_classes
result = combine_classes(container, "mx-auto", "px-4")
assert result == "container mx-auto px-4"

print("✅ SingleValueFactory tests passed!")

✅ SingleValueFactory tests passed!


## Directional Utilities

Support for utilities with directional variants (top, right, bottom, left):

In [ ]:
#| export
@dataclass
class Direction:
    """Represents a directional variant."""
    suffix: str
    css_suffix: str

In [ ]:
#| export
DIRECTIONS = { # Common directions
    "t": Direction("t", "top"),      # top
    "r": Direction("r", "right"),    # right
    "b": Direction("b", "bottom"),   # bottom
    "l": Direction("l", "left"),     # left
    "x": Direction("x", "inline"),   # horizontal
    "y": Direction("y", "block"),    # vertical
}

In [ ]:
#| export
class DirectionalUtility(StandardUtility):
    """Base class for utilities with directional variants."""
    
    def __init__(
        self,
        prefix: str,  # Base prefix (e.g., 'p' for padding)
        direction: Optional[str] = None  # Optional direction ('t', 'r', 'b', 'l', 'x', 'y')
    ):
        """Initialize with prefix and optional direction."""
        if direction and direction in DIRECTIONS:
            full_prefix = f"{prefix}{direction}"
        else:
            full_prefix = prefix
        super().__init__(full_prefix)

In [ ]:
# Test directional utilities
pt = DirectionalUtility("p", "t")
assert pt.build(4) == "pt-4"

px = DirectionalUtility("p", "x")
assert px.build(8) == "px-8"

## Negative Value Support

Support for utilities that can have negative values:

In [ ]:
#| export
class NegativeableUtility(StandardUtility):
    """Utility class that supports negative values."""
    
    def __init__(
        self,
        prefix: str,  # Base prefix
        negative: bool = False  # Whether this is a negative variant
    ):
        """Initialize with prefix and negative flag."""
        self.negative = negative
        full_prefix = f"-{prefix}" if negative else prefix
        super().__init__(full_prefix)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()